In [2]:
########################################################################
# import default python-library
########################################################################
import pickle
import os
import sys
import glob

import numpy as np
import librosa
import librosa.core
import librosa.feature
import yaml
import logging
import keras

from tqdm import tqdm
from sklearn import metrics
from keras.models import Model
from keras.layers import Input, Dense
########################################################################




########################################################################
# setup STD I/O
########################################################################
# 로깅을 설정하고 초기화하는 부분
logging.basicConfig(level=logging.DEBUG, filename="make_pretrain_v2.log")
logger = logging.getLogger(' ')
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
########################################################################




########################################################################
# file I/O
########################################################################
#파일 입출력 관련 함수 선언 부분
def save_pickle(filename, save_data):  
    logger.info("save_pickle -> {}".format(filename))
    with open(filename, 'wb') as sf:
        pickle.dump(save_data, sf)

def load_pickle(filename):
    logger.info("load_pickle <- {}".format(filename))
    with open(filename, 'rb') as lf:
        load_data = pickle.load(lf)
    return load_data


def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))


def demux_wav(wav_name, channel=0):
    try:
        multi_channel_data, sr = file_load(wav_name)
        if multi_channel_data.ndim <= 1:
            return sr, multi_channel_data
        return sr, np.array(multi_channel_data)[channel, :]
    except ValueError as msg:
        logger.warning(f'{msg}')
########################################################################







########################################################################
# feature extractor
########################################################################
#소리 파일 하나를 로그멜스펙트로그램 형태로 바꾸고 딥러닝 모델에 넣을 형태로 바꾸는 함수 
def file_to_vector_array(file_name, n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    sr, y = demux_wav(file_name)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)
    log_mel_spectrogram = 20.0 / power * np.log10(mel_spectrogram + sys.float_info.epsilon)
    vectorarray_size = len(log_mel_spectrogram[0, :]) - frames + 1
    if vectorarray_size < 1:
        return np.empty((0, dims), float)
    vectorarray = np.zeros((vectorarray_size, dims), float)
    for t in range(frames):
        vectorarray[:, n_mels * t: n_mels * (t + 1)] = log_mel_spectrogram[:, t: t + vectorarray_size].T
    return vectorarray


#소리 파일들의 이름명이 담긴 리스트를 입력하면 그것들을 하나의 데이터셋으로 합치는 함수
def list_to_vector_array(file_list, msg="calc...", n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, frames=frames, n_fft=n_fft, hop_length=hop_length, power=power)
        if idx == 0:
            dataset = np.zeros((vector_array.shape[0] * len(file_list), dims), float)
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array
    return dataset

# 예를 들어 machine type이 fan이고 id_가 id_00일때 id_00을 제외하고 fan 02, fan 04, fan 06으로 구성된 normal 소리음 데이터셋을 만드는 함수
def dataset_generator(target_dir, machine_type, id_, normal_dir_name="normal", ext="wav"):
    logger.info("target_dir : {}".format(target_dir))
    train_files = []
    machine_id = f"-6dB_{machine_type}"
    machine = f"{machine_type}"
    id_list = ["id_00", "id_02", "id_04", "id_06"]
    machine_type_path = os.path.join(target_dir, machine_id, machine)
    other_ids = [other_id for other_id in id_list if other_id != id_]
    for other_id in other_ids:
        other_machine_id = f"{other_id}"
        other_id_path = os.path.join(machine_type_path, other_machine_id, normal_dir_name)
        other_files = sorted(glob.glob(os.path.join(other_id_path, f"*.{ext}")))
        train_files.extend(other_files)    
    logger.info("train_file num : {num}".format(num=len(train_files)))
    return train_files
########################################################################




########################################################################
# keras model
########################################################################
#오토인코더 함수
def keras_model(inputDim):
    inputLayer = Input(shape=(inputDim,))
    h = Dense(64, activation="relu")(inputLayer)    
    h = Dense(64, activation="relu")(h)
    h = Dense(8, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(inputDim, activation=None)(h)
    return Model(inputs=inputLayer, outputs=h)
########################################################################




########################################################################
# main
########################################################################
# 메인 실행 부분
if __name__ == "__main__":
    with open("make_pretrain_v2.yaml", encoding='utf-8') as stream:
        pretrain_v2_param = yaml.safe_load(stream)

    # pickle데이터, model데이터, result데이터가 저장될 파일과 폴더들 관련 변수 선언
    os.makedirs(pretrain_v2_param["pickle_directory"], exist_ok=True)
    os.makedirs(pretrain_v2_param["model_directory"], exist_ok=True)
    
    pretrain_v2_data_dir = pretrain_v2_param["base_directory"]
    machine_types = ["fan", "valve", "slider", "pump"]
    id_list = ["id_00", "id_02", "id_04", "id_06"]
    print("\n===========================")

    #각 기계 타입 별, 해당 id를 제외한 나머지 id들의 normal 데이터 전체가 합쳐진 데이터를 생성하고 pretrain학습 시작 부분(위의 dataset_generator 함수 참고)
    for machine_type in machine_types:
        for machine_id in id_list:
            db = f"-6dB_{machine_type}"
            evaluation_result = {}
            train_pickle = "{pickle}/pretrain_except_{db}_{id_}.pickle".format(pickle=pretrain_v2_param["pickle_directory"],db=db, id_ = machine_id)
            
            #데이터셋 생성, pickle에 저장돼있다면 그것을 사용
            if os.path.exists(train_pickle):
                train_data = load_pickle(train_pickle)
            else:
                train_files = dataset_generator(pretrain_v2_data_dir, machine_type, machine_id)
                train_data = list_to_vector_array(train_files,
                                                  msg="generate train_dataset",
                                                  n_mels=pretrain_v2_param["feature"]["n_mels"],
                                                  frames=pretrain_v2_param["feature"]["frames"],
                                                  n_fft=pretrain_v2_param["feature"]["n_fft"],
                                                  hop_length=pretrain_v2_param["feature"]["hop_length"],
                                                  power=pretrain_v2_param["feature"]["power"])
                save_pickle(train_pickle, train_data)

            print("============== MODEL TRAINING ==============")
            
            model_directory = pretrain_v2_param["model_directory"]
            model_file = "{model}/pretrain_except_{machine_type}_{machine_id}.h5".format(model=model_directory,
                                                                                  machine_type=machine_type,
                                                                                  machine_id=machine_id)
            #모델 생성, 학습, 및 저장
            if not os.path.exists(model_file):                                                                      
                model = keras_model(pretrain_v2_param["feature"]["n_mels"] * pretrain_v2_param["feature"]["frames"])
                model.summary()
                model.compile(**pretrain_v2_param["fit"]["compile"])
                model.fit(train_data,
                                    train_data,
                                    epochs=pretrain_v2_param["fit"]["epochs"],
                                    batch_size=pretrain_v2_param["fit"]["batch_size"],
                                    shuffle=pretrain_v2_param["fit"]["shuffle"],
                                    validation_split=pretrain_v2_param["fit"]["validation_split"],
                                    verbose=pretrain_v2_param["fit"]["verbose"])
                model.save(model_file)
########################################################################

2023-08-26 00:56:24,407 - INFO - target_dir : ../Sample_data
2023-08-26 00:56:24,407 - INFO - target_dir : ../Sample_data
2023-08-26 00:56:24,412 - INFO - train_file num : 30
2023-08-26 00:56:24,412 - INFO - train_file num : 30


generate train_dataset: 100%|██████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 16.76it/s]
2023-08-26 00:56:26,215 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_fan_id_00.pickle
2023-08-26 00:56:26,215 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_fan_id_00.pickle


============== MODEL TRAINING ==============
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense (Dense)               (None, 64)                20544     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)

C:\Users\Baejs\anaconda3\envs\bae\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2023-08-26 00:56:31,190 - INFO - target_dir : ../Sample_data
2023-08-26 00:56:31,190 - INFO - target_dir : ../Sample_data
2023-08-26 00:56:31,192 - INFO - train_file num : 30
2023-08-26 00:56:31,192 - INFO - train_file num : 30
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 73.91it/s]
2023-08-26 00:56:31,605 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_fan_id_02.pickle
2023-08-26 00:56:31,605 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_fan_id_02.pickle


============== MODEL TRAINING ==============
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense_6 (Dense)             (None, 64)                20544     
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 8)                 520       
                                                                 
 dense_9 (Dense)             (None, 64)                576       
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Den

...

============== MODEL TRAINING ==============
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 320)]             0         
                                                                 
 dense_78 (Dense)            (None, 64)                20544     
                                                                 
 dense_79 (Dense)            (None, 64)                4160      
                                                                 
 dense_80 (Dense)            (None, 8)                 520       
                                                                 
 dense_81 (Dense)            (None, 64)                576       
                                                                 
 dense_82 (Dense)            (None, 64)                4160      
                                                                 
 dense_83 (De

2023-08-26 00:57:42,202 - INFO - target_dir : ../Sample_data
2023-08-26 00:57:42,202 - INFO - target_dir : ../Sample_data
2023-08-26 00:57:42,205 - INFO - train_file num : 30
2023-08-26 00:57:42,205 - INFO - train_file num : 30
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 85.45it/s]
2023-08-26 00:57:42,563 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_pump_id_04.pickle
2023-08-26 00:57:42,563 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_pump_id_04.pickle


============== MODEL TRAINING ==============
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 320)]             0         
                                                                 
 dense_84 (Dense)            (None, 64)                20544     
                                                                 
 dense_85 (Dense)            (None, 64)                4160      
                                                                 
 dense_86 (Dense)            (None, 8)                 520       
                                                                 
 dense_87 (Dense)            (None, 64)                576       
                                                                 
 dense_88 (Dense)            (None, 64)                4160      
                                                                 
 dense_89 (De

2023-08-26 00:57:47,261 - INFO - target_dir : ../Sample_data
2023-08-26 00:57:47,261 - INFO - target_dir : ../Sample_data
2023-08-26 00:57:47,263 - INFO - train_file num : 30
2023-08-26 00:57:47,263 - INFO - train_file num : 30
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 81.71it/s]
2023-08-26 00:57:47,636 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_pump_id_06.pickle
2023-08-26 00:57:47,636 - INFO - save_pickle -> ./pickle_pretrain_v2/pretrain_except_-6dB_pump_id_06.pickle


============== MODEL TRAINING ==============
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 320)]             0         
                                                                 
 dense_90 (Dense)            (None, 64)                20544     
                                                                 
 dense_91 (Dense)            (None, 64)                4160      
                                                                 
 dense_92 (Dense)            (None, 8)                 520       
                                                                 
 dense_93 (Dense)            (None, 64)                576       
                                                                 
 dense_94 (Dense)            (None, 64)                4160      
                                                                 
 dense_95 (De